# DOWNLOADS AND INSTALLS

In [1]:
import pandas as pd
import numpy as np
import os
import json
from openneuro import download
from helpers import *

/Users/justinhong/Documents/EEG_Research/EEGvenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DS_NUM = 'ds004789'
TASK = 'FR'
NUM = 1
MAX_SES = 12
DATA_PATH = f'../data/{TASK}/{TASK}{NUM}'

In [ ]:
# download participants info
download(dataset=DS_NUM, target_dir=DATA_PATH, include=['participants.tsv'])

In [3]:
participants = pd.read_csv(f'{DATA_PATH}/participants.tsv', sep='\t')

ids = participants['participant_id'].tolist()
print(f'There are {len(ids)} participants for task {TASK}{NUM}')

There are 280 participants for task FR1


In [4]:
SUBJECTS_RANGE = range(0, 280) # range to download (all subjects will be too much)
subjects = [ids[i] for i in SUBJECTS_RANGE]
bad_subs = ['sub-R1191J', 'sub-R1207J', 'sub-R1230J', 'sub-R1236J', 'sub-R1241J', 'sub-R1277J'] # doesn't have ieeg data

for bad_sub in bad_subs:
    if bad_sub in subjects:
        subjects.remove(bad_sub)

tsv_paths = [f'{sub}/ses-*/ieeg/*.tsv' for sub in subjects]
json_paths = [f'{sub}/ses-*/ieeg/*.json' for sub in subjects]

# FIX FOR FAULTY FOLDERS

In [15]:
# download data
download(dataset=DS_NUM, target_dir=DATA_PATH, include=tsv_paths+json_paths)


👋 Hello! This is openneuro-py 2024.2.0. Great to see you! 🤗

   👉 Please report problems 🤯 and bugs 🪲 at
      https://github.com/hoechenberger/openneuro-py/issues

🌍 Preparing to download ds004789 …


📁 Traversing directories for ds004789 : 351 entities [00:13, 24.42 entities/s]log_error - https://openneuro.org/crn/graphql: 502 Server Error: Bad Gateway for url: https://openneuro.org/crn/graphql
📁 Traversing directories for ds004789 : 351 entities [00:15, 24.42 entities/s]/Users/justinhong/Documents/EEG_Research/EEGvenv/lib/python3.11/site-packages/openneuro/_download.py:197: RuntimeWarning: coroutine 'sleep' was never awaited
  asyncio.sleep(retry_backoff)  # pyright: ignore[reportUnusedCoroutine]
📁 Traversing directories for ds004789 : 361 entities [00:15, 11.21 entities/s]

  Request timed out while fetching metadata, retrying …


📁 Traversing directories for ds004789 : 7944 entities [04:47, 27.59 entities/s]


RuntimeError: Could not find path in the dataset:
- sub-R1036M/ses-*/ieeg/*.tsv
There were no similar filenames found in the metadata. Please check your includes.

# SINGLE SUBJECT ANALYSIS

In [5]:
sub = subjects[0]
ses = 0
_ = get_sub_info(DATA_PATH, sub, ses, print_info=True)

Info for subject sub-R1001P session 0
------------------------------------------------------------
monopolar channel types + counts: {'ECOG': 66, 'SEEG': 22}
bipolar channel types + counts: {'ECOG': 54, 'SEEG': 18}
number of unique channel groups: 16
recording time: 1.305 hrs
sampling frequency: 500.0 Hz


In [6]:
sub = subjects[0]
ses = 1
_ = get_sub_info(DATA_PATH, sub, ses, print_info=True)

Info for subject sub-R1001P session 1
------------------------------------------------------------
monopolar channel types + counts: {'ECOG': 66, 'SEEG': 22}
bipolar channel types + counts: {'ECOG': 54, 'SEEG': 18}
number of unique channel groups: 16
recording time: 0.8528 hrs
sampling frequency: 500.0 Hz


# MULTI SUBJECT ANALYSIS

### Session-aggregated info per subject:

In [7]:
ses_agg_info = get_ses_agg_info(DATA_PATH, subjects, MAX_SES, print_info=True)

Session-aggregated info for 50 subjects for task FR1
------------------------------------------------------------
FR1 Subject sub-R1001P
{'sessions': 2, 'electrodes (mono)': {'ECOG': 66, 'SEEG': 22}, 'record time (hrs)': 2.1579, 'sample freq (Hz)': 500.0}

FR1 Subject sub-R1002P
{'sessions': 2, 'electrodes (mono)': {'ECOG': 58, 'SEEG': 16}, 'record time (hrs)': 1.8478, 'sample freq (Hz)': 500.0}

FR1 Subject sub-R1003P
{'sessions': 2, 'electrodes (mono)': {'ECOG': 92, 'SEEG': 8}, 'record time (hrs)': 1.6206, 'sample freq (Hz)': 500.0}

FR1 Subject sub-R1006P
{'sessions': 2, 'electrodes (mono)': {'ECOG': 99, 'SEEG': 24}, 'record time (hrs)': 1.5759, 'sample freq (Hz)': [500.0, 499.7071]}

FR1 Subject sub-R1010J
{'sessions': 2, 'electrodes (mono)': {'SEEG': 186}, 'record time (hrs)': 1.6783, 'sample freq (Hz)': 1000.0}

FR1 Subject sub-R1015J
{'sessions': 1, 'electrodes (mono)': {'SEEG': 180}, 'record time (hrs)': 0.5556, 'sample freq (Hz)': 1000.0}

FR1 Subject sub-R1018P
{'sessions': 1

### Subject and session-aggregated info over chosen subjects:

In [8]:
_ = get_sub_ses_agg_info(ses_agg_info, print_info=True)

Subject and session info aggregated over 275 subjects for task FR1:
------------------------------------------------------------
Number of sessions: 3014, avg 10.96 sessions per subject
Number of electrodes: {'ECOG': 2031, 'SEEG': 3459}, avg {'ECOG': 7.385454545454546, 'SEEG': 12.578181818181818} per subject (unchanging for each subject)
Recording time: 83.18 hrs, avg 0.30 hrs per subject, 0.03 hrs per session
